In [2]:
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode
import bcrypt

#### CONFIG

In [198]:
HOST = 'localhost'
DATABASE = 'am' # put your database
USER = 'root'
PASSWORD = 'root'
# SALT = bcrypt.gensalt() # generate salt for encrypting

In [199]:
# file = open('salt.txt', 'w')
# file.write(str(SALT)[2:-1])
# file.close()

In [224]:
file = open('salt.txt', 'r')
SALT = bytes(str(file.readline()), 'utf-8')

In [201]:
try:
    connection = mysql.connector.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute('select database();')
        record = cursor.fetchone()
        cursor.close()
        connection.close()
        print('Connected to {} database'.format(record[0]))
except Error as e:
    print('Error while connecting to MySQL', e)   

Connected to swe database


In [202]:
# test
def selectAll(tname):
    try:
        connection = mysql.connector.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute("select * from {};".format(tname))
            records = cursor.fetchall()
            print(cursor.rowcount)
            for row in records:
                print(row)
            cursor.close()
    except Error as e:
        print('Error while connecting to MySQL', e)   

#### CREATE TABLE IN DATABASE ```SWE```

In [203]:
DB_NAME = 'swe'

In [225]:
TABLES = {}

TABLES['users'] = ("""
    create table users (
        user_id decimal(6, 0) primary key,
        user_first_name varchar(20) not null,
        user_last_name varchar(30) not null,
        user_type varchar(10),
        user_gender varchar(6), -- gender: 0 - female, 1 - male
        user_email varchar(60) not null,
        user_password varchar(100) not null,
        user_enabled decimal(1, 0) default 0
    ); 
""")

TABLES['projects'] = ("""
    create table projects (
        id decimal(4, 0),
        project_id varchar(30),
        version varchar(2),
        primary key (project_id, version),
        title varchar(30),
        description varchar(4000) not null,
        status varchar(11) default 'NOT_STARTED',
        start_date date,
        expected_time decimal(38, 0),
        spent_time decimal(38, 0) default 0
    );
""")

TABLES['activities'] = ("""
    create table activities (
        activity_id decimal(8, 0) primary key,
        title varchar(30),
        description varchar(4000) not null,
        status varchar(11) default 'NOT_STARTED', -- status: 0 - not started, 1 - in process, 2 - finished
        start_date date,
        expected_time decimal(5, 0),
        spent_time decimal(5, 0) default 0
    );
""")

TABLES['report'] = ("""
    create table report (
        activity_id decimal(8, 0) not null,
        report_date date not null,
        primary key (activity_id, report_date),
        spent_time decimal(4, 0) not null
    );
""")

TABLES['consist'] = ("""
    create table consist (
        activity_id decimal(8, 0) primary key,
        project_id decimal(4, 0) not null
    );
""")

TABLES['role'] = ("""
    create table role (
        user_id decimal(6, 0) not null,  
        activity_id decimal(8, 0) not null,
        primary key (user_id, activity_id),
        position varchar(20)
    );
""")

TABLES['manage'] = ("""
    create table manage (
        user_id decimal(6, 0) not null,
        project_id decimal(4, 0) primary key
    );
""")

In [226]:
try:
    connection = mysql.connector.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
    if connection.is_connected():
        cursor = connection.cursor()
        for tname in TABLES:
            tdescription = TABLES[tname]
            try:
                print("Creating table {}: ".format(tname), end='')
                print(tname)
                cursor.execute(tdescription)
            except Error as err:
                if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                    print("already exists.")
                else:
                    print(err.msg)
            else:
                print("created.")
        cursor.close()
        connection.commit()
        connection.close()
except Error as e:
    print('Error while connecting to MySQL', e)   


Creating table users: users
already exists.
Creating table projects: projects
created.
Creating table activities: activities
already exists.
Creating table report: report
already exists.
Creating table consist: consist
already exists.
Creating table role: role
already exists.
Creating table manage: manage
already exists.


#### INSERT FUNCTION

In [206]:
def insert(data, tname):
    try:
        connection = mysql.connector.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
        if connection.is_connected():
            cursor = connection.cursor()
            for i in data:
                try:
                    cursor.execute(("insert into {} values (".format(tname)) + i + ");")
                except Error as err:
                    print(err.msg)
                else:
                    print("inserted.")
            cursor.close()
            connection.commit()
            connection.close()
    except Error as e:
        print('Error while connecting to MySQL', e)   

#### INSERT INTO TABLE ```USERS```

In [207]:
def crypt(password, salt=SALT):
    hpassword = bcrypt.hashpw(password, salt)
    return str(hpassword)[2:-1]

In [208]:
USERS = []

USERS += (
    """1, 'Sanzhar', 'Zhanibekov', 'EMPLOYEE', 'MALE', '190103039@stu.sdu.edu.kz', '{}', 1""".format(crypt(b'190103039')),
    """2, 'Dias', 'Aimuratov', 'EMPLOYEE', 'MALE', '190103268@stu.sdu.edu.kz', '{}', 1""".format(crypt(b'190103268')),
    """3, 'Asmina', 'Barkhandinova', 'MANAGER', 'FEMALE', '190103102@stu.sdu.edu.kz', '{}', 1""".format(crypt(b'190103102')),
    """4, 'Ilyas', 'Mohammad', 'EMPLOYEE', 'MALE', '190107005@stu.sdu.edu.kz', '{}', 1""".format(crypt(b'190107005')),
    """5, 'Ayan', 'Serikkhan', 'EMPLOYEE', 'FEMALE', '190103116@stu.sdu.edu.kz', '{}', 1""".format(crypt(b'190103116')),
    """6, 'Dauletkhan', 'Narynbaev', 'EMPLOYEE', 'FEMALE', '190107008@stu.sdu.edu.kz', '{}', 1""".format(crypt(b'190107008')),
    """7, 'Nurbakyt', 'Zhumatayev', 'EMPLOYEE', 'MALE', '190103092@stu.sdu.edu.kz', '{}', 1""".format(crypt(b'190103092')),
    """8, 'Madiyar', 'Akhmetkali', 'MANAGER', 'FEMALE', '190107081@stu.sdu.edu.kz', '{}', 1""".format(crypt(b'190107081')),
    """9, 'Nurzhanat', 'Zhubatyrov', 'EMPLOYEE', 'MALE', '190103418@stu.sdu.edu.kz', '{}', 1""".format(crypt(b'190103418'))
)

In [209]:
insert(USERS, 'users')

inserted.
inserted.
inserted.
inserted.
inserted.
inserted.
inserted.
inserted.
inserted.


#### INSERT INTO TABLE ```PROJECTS```

In [227]:
PROJECTS = []

PROJECTS += (
    """1, '1111DXP', 'A', 'Stock price prediction', '
        Stock market prediction is the act of trying to determine the future value 
        of a company stock or other financial instrument traded on an exchange. The 
        successful prediction of a stock''s future price could yield significant profit. 
        The efficient-market hypothesis suggests that stock prices reflect all currently 
        available information and any price changes that are not based on newly revealed 
        information thus are inherently unpredictable. Others disagree and those with 
        this viewpoint possess myriad methods and technologies which purportedly allow 
        them to gain future price information.
    ', 'FINISHED', '2022-01-01', 129600, 110253""",
    """2, '2222DXP', 'A', 'Activity monitoring', '
        The software house OnTimeSoftware develops software on contract,
        usually at fixed time, fixed price, and with specified functionality and
        quality. One of the hardest problems is to detect early on that an activity
        will take longer than expected. If you can detect it early, it is possible to
        help the developer currently carrying out the activity, for instance by letting
        a seasoned expert review the work and offer guidance. If you detect the
        problem late, the activity may have run off the track, be costly to correct,
        and it will soon hamper many other activities.
        Seasoned project managers know that they can get an early warning in
        this way: The developer responsible for a given activity records every day
        how many hours he has worked on the activity, and every weekend he gives
        an estimate of the remaining number of hours. Project management now
        computes an estimate of the total number of hours for the activity:
        total expected = hours spent + hours remaining
        In a straightforward activity this figure remains constant from week to
        week, and when no hours remain, it is finished. If the figure starts
        increasing, something is wrong, and help is most likely needed. Experience
        is that the problem in this way can be detected between 60% and 80% into
        the activity, while it otherwise tends to be detected when more than the total
        expected time has been used.
        Unfortunately, the procedure is cumbersome to manage, and company
        management wants to support it with an IT system. At the same time they
        want a better record of the work hours per activity, which many developers
        at present record many days too late. The recording is at present also used
        for billing some of the customers, and for collecting experience data of
        hours used for various sizes of projects.
        When staffing a project, department managers and project managers
        have troubles getting an overview of who will be available in the period
        where the project is carried out. They expect the system to give them an
        overview of this too.
    ', 'IN_PROCESS', '2022-03-22', 15480, 10000"""
)


In [228]:
insert(PROJECTS, 'projects')

inserted.
inserted.


#### INSERT INTO TABLE ```ACTIVITIES```

In [229]:
ACTIVITIES = []

ACTIVITIES += (
    """1, 'Data mining', '
        Data mining is a process used by companies to turn raw data into useful 
        information. By using software to look for patterns in large batches of 
        data, businesses can learn more about their customers to develop more 
        effective marketing strategies, increase sales and decrease costs.
    ', 'FINISHED', '2022-01-01', 14400, 14253""",
    """2, 'Data preparation', '
        Transforming raw data so that data scientists and analysts can run it 
        through machine learning algorithms to uncover insights or make predictions.
    ', 'FINISHED', '2022-01-11', 13000, 10080""",
    """3, 'Build model', '
        Organizations are implementing AI projects for numerous applications in a 
        wide range of industries. These applications include predictive analytics, 
        pattern recognition systems, autonomous systems, conversational systems, 
        hyper-personalization activities and goal-driven systems. Each of these 
        projects has something in common: They''re all predicated on an understanding of 
        the business problem and that data and machine learning algorithms must be applied 
        to the problem, resulting in a machine learning model that addresses the project''s needs.
    ', 'FINISHED', '2022-01-19', 30000, 28800""",
    """4, 'Testing model', '
        One of the aspects of building a Machine Learning model is to check whether the data 
        used for training and testing the model belong to an adversary dataset. The adversary 
        data sets are that can be used to skew the results of the model by training the model 
        using incorrect data called as Data Poisoning Attack.
    ', 'FINISHED', '2022-02-09', 24680, 21600""",
    """5, 'Debugging', '
        Debugging is the process of detecting and removing of existing and potential errors 
        (also called as ''bugs'') in a software code that can cause it to behave unexpectedly 
        or crash. To prevent incorrect operation of a software or system, debugging is used 
        to find and resolve bugs or defects.
    ', 'FINISHED', '2022-02-26', 21600, 14400""",
    """6, 'Validation', '
        Validation data provides an initial check that the model can return useful predictions 
        in a real-world setting, which training data cannot do. The ML algorithm can assess 
        training data and validation data at the same time.
    ', 'FINISHED', '2022-02-08', 22600, 21120""",

    """7, 'Create database', '
        A database is an organized collection of structured information, or data, typically stored 
        electronically in a computer system. A database is usually controlled by a database management 
        system (DBMS).
    ', 'FINISHED', '2022-04-01', 1440, 960""",
    """8, 'Login Page Front', '
        Frontend login is the page that your registered site users use to login when you require 
        authentication for your site. For example, you may want all users who can leave comments to be 
        logged in; or you can require everybody who checks out products to be registered and logged in.
    ', 'FINISHED', '2022-04-03', 1440, 960""",
    """9, 'Login Page Back', '
        The back-end, also called the server side, consists of the server which provides data on request, 
        the application which channels it, and the database which organizes the information. For example, 
        when a customer browses shoes on a website, they are interacting with the front end.
    ', 'FINISHED', '2022-04-03', 1440, 960""",
    """10, 'Connect to database', '
        A database connection is a facility in computer science that allows client software to talk to 
        database server software, whether on the same machine or not. A connection is required to send 
        commands and receive answers, usually in the form of a result set. Connections are a key concept 
        in data-centric programming.
    ', 'FINISHED', '2022-04-05', 600, 480""",
    """11, 'User Dashboard Page Front', '
        A front-end developer builds the front-end portion of websites and web applications—that is, 
        the part that users actually see and interact with. A front-end developer creates websites and 
        applications using web languages such as HTML, CSS, and JavaScript allow users to access and 
        interact with the site or app.
    ', 'FINISHED', '2022-04-06', 1440, 960""",
    """12, 'User Dashboard Page Back', '
        When back-end developers create apps that render on the server-side, they use the same building 
        blocks as front-end developers: HTML, CSS and JavaScript. Back-end developers also work with 
        software stacks that include operating systems, web servers, frameworks, languages, programming APIs and more.
    ', 'FINISHED', '2022-04-06', 1440, 960""",
    """13, 'Create project pane Front', '
       In publishing and graphic design, Lorem ipsum is a placeholder text commonly used to demonstrate the visual 
       form of a document or a typeface without relying on meaningful content. Lorem ipsum may be used as 
       a placeholder before the final copy is available.
    ', 'IN_PROCESS', '2022-04-10', 960, 300""",
    """14, 'Create project pane Back', '
       In publishing and graphic design, Lorem ipsum is a placeholder text commonly used to demonstrate the visual 
       form of a document or a typeface without relying on meaningful content. Lorem ipsum may be used as 
       a placeholder before the final copy is available.
    ', 'IN_PROCESS', '2022-04-10', 960, 120""",
    """15, 'Make Analysis', '
       Data analysis is defined as a process of cleaning, transforming, and modeling data to discover useful 
       information for business decision-making. The purpose of Data Analysis is to extract useful information 
       from data and taking the decision based upon the data analysis
    ', 'NOT_STARTED', NULL, 2880, 0""",
    """16, 'Make Documentation', '
       Documentation is any communicable material that is used to describe, explain or instruct regarding some 
       attributes of an object, system or procedure, such as its parts, assembly, installation, maintenance and 
       use. Documentation can be provided on paper, online, or on digital or analog media, such as audio tape or CDs.
    ', 'NOT_STARTED', NULL, 2880, 0""",
)

In [230]:
insert(ACTIVITIES, 'activities')

Duplicate entry '1' for key 'activities.PRIMARY'
Duplicate entry '2' for key 'activities.PRIMARY'
Duplicate entry '3' for key 'activities.PRIMARY'
Duplicate entry '4' for key 'activities.PRIMARY'
Duplicate entry '5' for key 'activities.PRIMARY'
Duplicate entry '6' for key 'activities.PRIMARY'
Duplicate entry '7' for key 'activities.PRIMARY'
Duplicate entry '8' for key 'activities.PRIMARY'
Duplicate entry '9' for key 'activities.PRIMARY'
Duplicate entry '10' for key 'activities.PRIMARY'
Duplicate entry '11' for key 'activities.PRIMARY'
Duplicate entry '12' for key 'activities.PRIMARY'
Duplicate entry '13' for key 'activities.PRIMARY'
Duplicate entry '14' for key 'activities.PRIMARY'
Duplicate entry '15' for key 'activities.PRIMARY'
Duplicate entry '16' for key 'activities.PRIMARY'


#### CONNECT ACTIVITY TO PROJECT IN TABLE ```CONSIST```

In [214]:
CONNECT1 = []

CONNECT1 += (
    "1, 1",
    "2, 1",
    "3, 1",
    "4, 1",
    "5, 1",
    "6, 1",

    "7, 2",
    "8, 2",
    "9, 2",
    "10, 2",
    "11, 2",
    "12, 2",
    "13, 2",
    "14, 2",
    "15, 2",
    "16, 2",
)

In [215]:
insert(CONNECT1, 'consist')

Duplicate entry '1' for key 'consist.PRIMARY'
Duplicate entry '2' for key 'consist.PRIMARY'
Duplicate entry '3' for key 'consist.PRIMARY'
Duplicate entry '4' for key 'consist.PRIMARY'
Duplicate entry '5' for key 'consist.PRIMARY'
Duplicate entry '6' for key 'consist.PRIMARY'
Duplicate entry '7' for key 'consist.PRIMARY'
Duplicate entry '8' for key 'consist.PRIMARY'
Duplicate entry '9' for key 'consist.PRIMARY'
Duplicate entry '10' for key 'consist.PRIMARY'
Duplicate entry '11' for key 'consist.PRIMARY'
Duplicate entry '12' for key 'consist.PRIMARY'
Duplicate entry '13' for key 'consist.PRIMARY'
Duplicate entry '14' for key 'consist.PRIMARY'
Duplicate entry '15' for key 'consist.PRIMARY'
Duplicate entry '16' for key 'consist.PRIMARY'


#### CONNECT USERS(MANAGERS) TO PROJECT IN TABLE ```MANAGE```

In [216]:
CONNECT2 = []

CONNECT2 += (
    "8, 1",
    "3, 2"
)

In [217]:
insert(CONNECT2, 'manage')

Duplicate entry '1' for key 'manage.PRIMARY'
Duplicate entry '2' for key 'manage.PRIMARY'


#### CONNECT USERS(EMPLOYEES) TO ACTIVITY IN TABLE ```ROLE```

In [218]:
CONNECT3 = []

CONNECT3 += (
    "6, 1, 'Data engineer'",
    "6, 2, 'Data engineer'", 
    "6, 3, 'Data engineer'",
    "9, 4, 'QA engineer'",
    "7, 5, 'Debugger'",
    "9, 6, 'QA engineer'",

    "2, 7, 'Data engineer'",
    "5, 8, 'Front-End developer'",
    "1, 9, 'Back-End developer'",
    "2, 10, 'Data engineer'",
    "5, 11, 'Front-End developer'",
    "1, 12, 'Back-End developer'",
    "5, 13, 'Front-End developer'",
    "1, 14, 'Back-End developer'"
)

In [219]:
insert(CONNECT3, 'role')

Duplicate entry '6-1' for key 'role.PRIMARY'
Duplicate entry '6-2' for key 'role.PRIMARY'
Duplicate entry '6-3' for key 'role.PRIMARY'
Duplicate entry '9-4' for key 'role.PRIMARY'
Duplicate entry '7-5' for key 'role.PRIMARY'
Duplicate entry '9-6' for key 'role.PRIMARY'
Duplicate entry '2-7' for key 'role.PRIMARY'
Duplicate entry '5-8' for key 'role.PRIMARY'
Duplicate entry '1-9' for key 'role.PRIMARY'
Duplicate entry '2-10' for key 'role.PRIMARY'
Duplicate entry '5-11' for key 'role.PRIMARY'
Duplicate entry '1-12' for key 'role.PRIMARY'
Duplicate entry '5-13' for key 'role.PRIMARY'
Duplicate entry '1-14' for key 'role.PRIMARY'


#### INSERT INTO TABLE ```REPORT```

In [220]:
REPORTS = []

REPORTS += (
    "7, '2022-04-01', 480",
    "7, '2022-04-02', 480",
    "8, '2022-04-03', 480",
    "8, '2022-04-04', 480",
    "9, '2022-04-03', 480",
    "9, '2022-04-04', 480",
    "10, '2022-04-05', 480",
    "11, '2022-04-06', 480",
    "11, '2022-04-07', 480",
    "12, '2022-04-06', 480",
    "12, '2022-04-07', 480",
    "13, '2022-04-10', 300",
    "14, '2022-04-10', 120",
)

In [221]:
insert(REPORTS, 'report')

Duplicate entry '7-2022-04-01' for key 'report.PRIMARY'
Duplicate entry '7-2022-04-02' for key 'report.PRIMARY'
Duplicate entry '8-2022-04-03' for key 'report.PRIMARY'
Duplicate entry '8-2022-04-04' for key 'report.PRIMARY'
Duplicate entry '9-2022-04-03' for key 'report.PRIMARY'
Duplicate entry '9-2022-04-04' for key 'report.PRIMARY'
Duplicate entry '10-2022-04-05' for key 'report.PRIMARY'
Duplicate entry '11-2022-04-06' for key 'report.PRIMARY'
Duplicate entry '11-2022-04-07' for key 'report.PRIMARY'
Duplicate entry '12-2022-04-06' for key 'report.PRIMARY'
Duplicate entry '12-2022-04-07' for key 'report.PRIMARY'
Duplicate entry '13-2022-04-10' for key 'report.PRIMARY'
Duplicate entry '14-2022-04-10' for key 'report.PRIMARY'
